In [1]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections

from PyHEADTAIL_feedback.feedback import OneboxFeedback
from PyHEADTAIL_feedback.derived_processors import Bypass
from PyHEADTAIL_feedback.processors import BypassLinearTransform, BypassMultiplication, BypassAddition

np.random.seed(0)

PyHEADTAIL v1.9.4


